In [37]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

'\nfunction ClickConnect(){\nconsole.log("Working"); \ndocument.querySelector("colab-toolbar-button#connect").click() \n}\nsetInterval(ClickConnect,60000)\n'

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
!cat /proc/meminfo

MemTotal:       26751700 kB
MemFree:          342384 kB
MemAvailable:   16535464 kB
Buffers:           32876 kB
Cached:           373596 kB
SwapCached:            0 kB
Active:         14370120 kB
Inactive:       11714496 kB
Active(anon):    9608828 kB
Inactive(anon):      368 kB
Active(file):    4761292 kB
Inactive(file): 11714128 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               160 kB
Writeback:             0 kB
AnonPages:      25678428 kB
Mapped:           205636 kB
Shmem:               928 kB
Slab:             136456 kB
SReclaimable:      82196 kB
SUnreclaim:        54260 kB
KernelStack:        4512 kB
PageTables:        58608 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375848 kB
Committed_AS:   29841084 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1840 kB
AnonHugePages:   

In [40]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [41]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("GSAR")

R[write to console]: Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

R[write to console]: Installing package(s) 'GSAR'

R[write to console]: trying URL 'https://bioconductor.org/packages/3.10/bioc/src/contrib/GSAR_1.20.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2185168 bytes (2.1 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console

In [42]:
%%R
library(tidyverse)
library(MASS)
library(GSAR)
library(parallel)

In [43]:
%%R

# Simplex sample generator
simplex_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n-1,0,1)  %>%  
                    sort(decreasing=FALSE)
        sample  <- append(0.0, sample)  %>% 
                    append(1.0)
        sample  <- sample[-1] - sample
        sample  <- sample[1:n]

        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# Genberate hypercube
hyper_gen  <- function(n, samples){
    simplex_out  <- matrix(, nrow = samples, ncol = n)
    for(r in c(1:samples)){
        sample  <- runif(n,-1,1) 
        sample  <- sample[1:n]
        simplex_out[r,]  <- sample
     }
    return(simplex_out)
}

# MST for simplex
mst <- function(file_, nsamp=10000, perm=50){
    # Read files with samples
    df_  <- read_csv(file_)  %>%  
    dplyr::select(starts_with('x_'))  %>% 
    sample_n(size=nsamp, replace=FALSE)
    # Get dim of the read file
    n_rows  <- dim(df_)[1]
    n_cols  <- dim(df_)[2]
        if (n_rows > (nsamp-1)){
        simplex_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
        
        df_  <- df_  %>% 
                data.matrix()   %>% 
                unname()
        
        df_  <- rbind(df_, simplex_sample)  
        df_  <- df_ %>% 
                aperm(c(2,1)) 
        result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                         nperm=perm, pvalue.only=FALSE)
        z_df  <- tibble(dim=n_cols, z=result$statistic, 
                        p_value=result$p.value,
                             nsamp=nsamp)
        gc()
        return(z_df)
        
    }
    
}


In [44]:
%%R
path  <- '/content/drive/My Drive/mhar_paper_samples_2/'
set.seed(5)
files <- list.files(path)
files  <- paste(path, files, sep='')

# Extract dimenions
m <- regexpr("%[0-9]+.csv", files, perl=TRUE)
dimension <- regmatches(files, m)
m <- regexpr("[0-9]+", dimension, perl=TRUE)
dimension <- regmatches(dimension, m)
dimension

#Extract figure
m <- regexpr("[0-9]+_[a-z]+%", files, perl=TRUE)
figure <- regmatches(files, m)
m <- regexpr("[a-z]+", figure, perl=TRUE)
figure <- regmatches(figure, m)
figure

# Extract seed
m <- regexpr("/[0-9]+_", files, perl=TRUE)
sed <- regmatches(files, m)
m <- regexpr("[0-9]+", sed, perl=TRUE)
sed <- regmatches(sed, m)

df_simulations <- tibble(file=files, n=dimension, figure=figure, seed=sed)

In [45]:
%%R
df_simulations

# A tibble: 81 x 4
   file                                                     n     figure   seed 
   <chr>                                                    <chr> <chr>    <chr>
 1 /content/drive/My Drive/mhar_paper_samples_2/100_hyperc… 3     hypercu… 100  
 2 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 15    hypercu… 1064 
 3 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 25    hypercu… 1064 
 4 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 3     hypercu… 1064 
 5 /content/drive/My Drive/mhar_paper_samples_2/1064_hyper… 50    hypercu… 1064 
 6 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 15    simplex  1064 
 7 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 25    simplex  1064 
 8 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 3     simplex  1064 
 9 /content/drive/My Drive/mhar_paper_samples_2/1064_simpl… 50    simplex  1064 
10 /content/drive/My Drive/mhar_paper_samples_2/1197_hyper… 15    hypercu… 1197 
# … with 

In [ ]:
%%R

save_path <- '/drive/My Drive/mst_5000/p100_mst_first_7000.csv'

# Create random seeds
seeds_ <- sample(1:10000000, 1000, replace=T)
s<- 1


## PArse existing experiments
if(!file.exists(save_path)){
    existance<- tibble(figure=character(),dim=numeric(), z=numeric(), 
                          p_value=numeric(),
                              nsamp=numeric(), file=character(), perm=numeric(),
               seed=numeric()) 
    
}else{
    existance <- read.csv(save_path) %>% dplyr::select( -contains('X'))
}

z_df <- existance

for(f in files){
  # Set hyperparameters
  set.seed(seeds_[s])
  file_name <- f
  perm <- 100
  n_sample <-7000
  print(file_name)
  df_  <- read_csv(file_name)
  df_ <- df_[c(1:n_sample),]
  # Get dim of the read file
  n_rows  <- dim(df_)[1]
  n_cols  <- dim(df_)[2]
  print(n_rows)
  print(n_cols)

  # Check if the experiment already runned
  check_df <- FALSE
  if(f %in% existance$f){
  check_df <- existance %>% filter(file==f)
  check_df <- (check_df$dim == n_cols) && (check_df$nsamp == n_sample)  && (check_df$perm == perm)
  }

  check_df <- !check_df

  if(check_df){
    if (grepl('hyper', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
    typ <- 'Hypercube'
  }
  if (grepl('simplex', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
    typ <- 'Simplex'
  }

  df_  <- df_  %>% data.matrix()   %>% unname()
          
  df_  <- rbind(df_, real_sample)  
  df_  <- df_ %>% aperm(c(2,1)) 
  dim(df_)
  result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                          nperm=2, pvalue.only=FALSE)
  z_df  <- tibble(figure=typ,dim=n_cols, z=result$statistic, 
                          p_value=result$p.value,
                              nsamp=n_sample, file=f, perm=perm, 
                  seed=seeds_[s]) %>% rbind(z_df)
  z_df %>% write.csv(save_path)
  gc()  
  
  }
  #Increment seed index
  s <- s + 1 
}




[1] "/content/drive/My Drive/mhar_paper_samples_2/100_hypercube%1000%3.csv"


R[write to console]: Parsed with column specification:
cols(
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double()
)



[1] 7000
[1] 3


In [ ]:
%%R

save_path <- '/content/drive/My Drive/mst_5000/p100_mst_last_7000.csv'

# Create random seeds
seeds_ <- sample(1:10000000, 1000, replace=T)
s<- 1


## PArse existing experiments
if(!file.exists(save_path)){
    existance<- tibble(figure=character(),dim=numeric(), z=numeric(), 
                          p_value=numeric(),
                              nsamp=numeric(), file=character(), perm=numeric(),
               seed=numeric()) 
    
}else{
    existance <- read.csv(save_path) %>% dplyr::select( -contains('X'))
}

z_df <- existance

for(f in files){
  # Set hyperparameters
  set.seed(seeds_[s])
  file_name <- f
  perm <- 100
  n_sample <-7000
  print(file_name)
  df_  <- read_csv(file_name)
  df_ <- df_[c(3000:10000),]
  # Get dim of the read file
  n_rows  <- dim(df_)[1]
  n_cols  <- dim(df_)[2]
  print(n_rows)
  print(n_cols)

  # Check if the experiment already runned
  check_df <- FALSE
  if(f %in% existance$f){
  check_df <- existance %>% filter(file==f)
  check_df <- (check_df$dim == n_cols) && (check_df$nsamp == n_sample)  && (check_df$perm == perm)
  }

  check_df <- !check_df

  if(check_df){
    if (grepl('hyper', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(hyper_gen(n_cols,n_rows))
    typ <- 'Hypercube'
  }
  if (grepl('simplex', f, fixed = TRUE)){
    real_sample  <- suppressWarnings(simplex_gen(n_cols,n_rows))
    typ <- 'Simplex'
  }

  df_  <- df_  %>% data.matrix()   %>% unname()
          
  df_  <- rbind(df_, real_sample)  
  df_  <- df_ %>% aperm(c(2,1)) 
  dim(df_)
  result <- WWtest(object=df_, group=c(rep(1, n_rows),rep(2, n_rows)),
                          nperm=2, pvalue.only=FALSE)
  z_df  <- tibble(figure=typ,dim=n_cols, z=result$statistic, 
                          p_value=result$p.value,
                              nsamp=n_sample, file=f, perm=perm, 
                  seed=seeds_[s]) %>% rbind(z_df)
  z_df %>% write.csv(save_path)
  gc()  
  
  }
  #Increment seed index
  s <- s + 1 
}


